In [9]:
### Basic Setup

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import cross_val_score

%matplotlib inline

pd.options.display.max_columns = 0

In [10]:
train = pd.read_csv('../cleaned_transformed/train.csv')
test = pd.read_csv('../cleaned_transformed/test.csv')
orig_test = pd.read_csv('../Kaggle_Info/test.csv')
X = train.iloc[:, :-1]
y = train.iloc[:, -1]
X.head()

,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,1stFlrSF,FullBath,YearsBeforeSold,ExterQual_TA,BsmtQual_Ex,KitchenQual_TA,GarageFinish_Unf,KitchenQual_Ex
0,7,1710,2,856,856,2,5,0,0,0,0,0
1,6,1262,2,1262,1262,2,31,1,0,1,0,0
2,7,1786,2,920,920,2,6,0,0,0,0,0
3,7,1717,3,756,961,1,36,1,0,0,1,0
4,8,2198,3,1145,1145,2,8,0,0,0,0,0


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

prep_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(2))
])
X_poly = prep_pipe.fit_transform(X)
lr = LinearRegression()
# rfe = RFECV(lr, 3, cv=10)
rfe = RFE(lr, 18)
rfe.fit(X_poly, y)

X_poly = X_poly[:, rfe.support_]
rmses = np.sqrt(-cross_val_score(lr, X_poly, y, cv=10, scoring='neg_mean_squared_error'))
(np.mean(rmses), np.std(rmses))

lr.fit(X_poly, y)
lr_preds = lr.predict(X_poly)
rmse = mean_squared_error(y, lr_preds)**.5
rmse

test_poly = prep_pipe.transform(test)
lr_test_pred = lr.predict(test_poly[:, rfe.support_])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=28, n_estimators=26)
rf.fit(X, y)
rf_preds = rf.predict(X)
rmse = mean_squared_error(y, rf_preds)**.5
rmse

rf_test_pred = rf.predict(test)

In [ ]:
comb_preds = (lr_test_pred + rf_test_pred) / 2.

*LinearRegression* with polynomial features and *RandomForest* look like the most accurate regression models so we make an ensemble of them.